In [1]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Times Higher Education

С этим рейтингом тоже создаём объект `session`.

In [2]:
session = requests.session()
session.headers.update({
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
})

Дальше собираем данные по годам.

In [4]:
years = range(2020, 2023 + 1)
url_tpl = 'https://www.timeshighereducation.com/world-university-rankings/{year}/world-ranking'
ranking = pd.DataFrame(columns=['year'])

for year in years:
    html_url = url_tpl.format(year=year)
    html_resp = session.get(html_url)

    if html_resp.status_code == 200:
        html = html_resp.text
        soup = BeautifulSoup(html)
        scripts = soup.find_all('script')
        search_words = 'init_drupal_core_settings'
        drupal_settings_str = None

        for script in scripts:
            script_text = script.text

            if script_text.find(search_words) > -1:
                drupal_settings_str = script_text

        if drupal_settings_str:
            found = re.search(f"world_university_rankings_{year}_0__[\w\d]+\.json", drupal_settings_str)

            if found:
                data_url = f'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/{found[0]}'
                data_resp = session.get(data_url)

                if data_resp.status_code == 200:
                    data = data_resp.json()
                    chunk = pd.json_normalize(data['data'])
                    chunk['year'] = year
                    ranking = pd.concat([ranking, chunk], ignore_index=True)
                else:
                    print(f"{data_url} return status code {data_resp.status_code}")
    else:
        print(f"{html_url} return status code {html_resp.status_code}")

    # Бережём источник данных
    time.sleep(1)

ranking.to_csv(f"data/the_{min(years)}-{max(years)}.csv", index=False)
ranking.to_excel(f"data/the_{min(years)}-{max(years)}.xlsx", index=False)
ranking.head()

/tmp/ipykernel_372402/2402199818.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)
/tmp/ipykernel_372402/2402199818.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)
/tmp/ipykernel_372402/2402199818.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)


,year,rank_order,rank,name,scores_overall,scores_overall_rank,scores_teaching,scores_teaching_rank,scores_research,scores_research_rank,...,stats_pc_intl_students,stats_female_male_ratio,aliases,subjects_offered,closed,unaccredited,disabled,apply_link,cta_button.link,cta_button.text
0,2020,10,1,University of Oxford,95.4,10,90.5,6,99.6,1,...,41%,46 : 54,University of Oxford,"Accounting & Finance,General Engineering,Commu...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
1,2020,20,2,California Institute of Technology,94.5,20,92.1,2,97.2,4,...,30%,34 : 66,California Institute of Technology caltech,"Languages, Literature & Linguistics,Economics ...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
2,2020,30,3,University of Cambridge,94.4,30,91.4,4,98.7,2,...,37%,47 : 53,University of Cambridge,"Business & Management,General Engineering,Art,...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
3,2020,40,4,Stanford University,94.3,40,92.8,1,96.4,5,...,23%,43 : 57,Stanford University,"Physics & Astronomy,Computer Science,Politics ...",False,False,False,NaN,https://www.timeshighereducation.com/student/r...,Admissions Support
4,2020,50,5,Massachusetts Institute of Technology,93.6,50,90.5,5,92.4,10,...,34%,39 : 61,Massachusetts Institute of Technology,"Mathematics & Statistics,Languages, Literature...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support


Из полученных данных соберём количество 🇷🇺 российских университетов в рейтинге по годам.

In [5]:
ranking[ranking['location'] == 'Russian Federation'][['year', 'location']].\
    groupby(by='year').count().rename(columns={'location': 'count'})

,count
year,
2020,39
2021,48
2022,100
2023,103
